In [ ]:
# Installations
!pip install gzip
!pip install geopy
!pip install boto

In [1]:
#jsonlines code
# Based on:
import jsonlines
import pprint
import json
import numpy as np
import pandas as pd
import gzip
import os

In [ ]:
#Get all relevant folders across all subdirectories
dir_name = 'D:\Dropbox\Programming\COVID_Religion_Project\COVID-19-TweetIDs\gz_unzip_test'
extension = ".gz"

#os.chdir(dir_name) # change directory from working dir to dir with files

for item in os.listdir(dir_name): # loop through items in dir
    if item.endswith(extension): # check for ".zip" extension
        print(item)
        file_name = os.path.abspath(item) # get full path of files
        zip_ref = zipfile.ZipFile(file_name) # create zipfile object
        zip_ref.extractall(dir_name) # extract file to dir
        zip_ref.close() # close file
        os.remove(file_name) # delete zipped file

In [ ]:
#for later upgrade to deal with gzip files, rather than just jsonl
file = 'D:\Dropbox\Programming\COVID_Religion_Project\COVID-19-TweetIDs\gz_unzip_test\coronavirus-tweet-id-2020-01-21-22.jsonl.gz'
f = gzip.open(file, 'rb')
file_content = f.readlines()
f.close()

#Create list of dictionaries
final_json = [json.loads(line.decode('utf8')) for line in file_content]


In [36]:
def jsonl_to_jsonlist(link):
    """
    Takes in a compressed json file, returns a list of json dictionary objects
    Deletes the entitites and retweeted_status keys in the dictionary whenever present
    Also removes the 'entities' subdictionary to limit the depth of nesting to 1
    """
    json_list =[]
    with jsonlines.open(link) as f:
        for line in f.iter():
            try:
                del line['entities']
            except:
                pass            
            
            try:
                del line['retweeted_status']
            except:
                #since some tweets were never retweeted
                pass
            #we don't want unnecessary multi-level nesting
            for key in line['user'].keys():
                if isinstance(line['user'][key], list):
                    line['user'][key] = line['user'][key][0]
            
            json_list.append(line)
    return json_list

In [3]:
#Taking sample files from the beginning of Corona cases (March) and the present day
mar_link = 'COVID-19-TweetIDs\gz_unzip_test\coronavirus-tweet-id-2020-03-08-17.jsonl'
jul_link = 'COVID-19-TweetIDs\gz_unzip_test\coronavirus-tweet-id-2020-07-01-05.jsonl'

## Understanding the Shape of the Data

In [ ]:
jul_json = jsonl_to_jsonlist(jul_link)
test = jul_json[3]
pprint.pprint(test)

In [40]:
mar_json = jsonl_to_jsonlist(mar_link)

In [38]:
test = mar_json[39]
pprint.pprint(test)

{'contributors': None,
 'coordinates': None,
 'created_at': 'Sun Mar 08 17:02:33 +0000 2020',
 'display_text_range': [0, 140],
 'favorite_count': 0,
 'favorited': False,
 'full_text': 'RT @itsJeffTiedrich: holy fucking shit, over at Facebook some '
              'deplorable is screaming at me that the coronavirus is a '
              '"Democrat hoax" and t…',
 'geo': None,
 'id': 1236698835579072500,
 'id_str': '1236698835579072513',
 'in_reply_to_screen_name': None,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'is_quote_status': False,
 'lang': 'en',
 'place': None,
 'retweet_count': 3818,
 'retweeted': False,
 'source': '<a href="http://twitter.com/download/iphone" '
           'rel="nofollow">Twitter for iPhone</a>',
 'truncated': False,
 'user': {'contributors_enabled': False,
          'created_at': 'Tue Jul 02 00:29:33 +0000 2019',
          'default_profile': True,
          'default_profile_

In [47]:
cols = list(test.keys())
cols.remove('user')
print(len(cols))
print(cols)

22
['created_at', 'id', 'id_str', 'full_text', 'truncated', 'display_text_range', 'source', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'geo', 'coordinates', 'place', 'contributors', 'is_quote_status', 'retweet_count', 'favorite_count', 'favorited', 'retweeted', 'lang']


In [48]:
#normalizing json
# https://stackoverflow.com/questions/40588852/pandas-read-nested-json
from pandas.io.json import json_normalize 
df_normal = json_normalize(mar_json,
                           record_path=['user'],
                           meta=cols,
                           #record_prefix='user_',
                           #meta_prefix='tweet_',
                           errors='ignore')

MemoryError: 

In [45]:
df_normal

,0
0,id
1,id_str
2,name
3,screen_name
4,location
5,description
6,url
7,entities
8,protected
9,followers_count


In [44]:
mar_json

[{'created_at': 'Sun Mar 08 17:02:33 +0000 2020',
  'id': 1236698836237799400,
  'id_str': '1236698836237799427',
  'full_text': "RT @thedailybeast: #SNL brutally roasts Trump’s #coronavirus response: “Oh my god, we're all gonna die.” https://t.co/GtcTgkBEDh",
  'truncated': False,
  'display_text_range': [0, 128],
  'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>',
  'in_reply_to_status_id': None,
  'in_reply_to_status_id_str': None,
  'in_reply_to_user_id': None,
  'in_reply_to_user_id_str': None,
  'in_reply_to_screen_name': None,
  'user': {'id': 838740166286184400,
   'id_str': '838740166286184448',
   'name': 'Esqueuezeme ⚓ maskmaking Navy Vet',
   'screen_name': 'Esqueuezme',
   'location': 'Cleaning up dog/kid mess, USA',
   'description': 'Disabled Veteran, parent, dog rescuer, resister, atheist, lousy at bios. Add to a list/fol 2 sell me crap=block.♀️ #vetsresistsquadron Need a mask? DM me.',
   'url': None,
   'entities': {'des

In [ ]:
len(df_normal.columns)

# Analyzing Tweet Content

In [25]:
import pandas as pd
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import nltk
import string

# Some Functions from Last Time to get us started:
def get_wordnet_pos(word):

    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": nltk.corpus.wordnet.ADJ,
                "N": nltk.corpus.wordnet.NOUN,
                "V": nltk.corpus.wordnet.VERB,
                "R": nltk.corpus.wordnet.ADV}

    return tag_dict.get(tag, nltk.corpus.wordnet.NOUN)

def get_lemmas(text):
    stop = nltk.corpus.stopwords.words('english') + list(string.punctuation)
    tokens = [i for i in nltk.word_tokenize(text.lower()) if i not in stop]
    lemmas = [nltk.stem.WordNetLemmatizer().lemmatize(t, get_wordnet_pos(t)) for t in tokens]
    return lemmas

def get_tokens(text):
    # drop punctuation, but keep stopwords for initial word counting
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = [i for i in nltk.word_tokenize(text.lower())]
    return tokens

def fill_topic_weights(df_row, bow_corpus):
    # Fill topic weights for topics in songs
    try:
        for i in ldamodel[bow_corpus[df_row.name]]:
            df_row[str(i[0])] = i[1]
    except:
        return df_row
    return df_row


In [ ]:
# Topic Modelling

def top_songs_by_topic(ldamodel, corpus, ntop=1):
    #Returns empty dictionary
    topn_songs_by_topic = {}
    for i in range(len(ldamodel.print_topics())):
        # For each topic, collect the most representative song(s) (i.e. highest probability containing words belonging to topic):
        top = sorted(zip(range(len(corpus)), ldamodel[corpus]), reverse=True, key=lambda x: abs(dict(x[1]).get(i, 0.0)))
        topn_songs_by_topic[i] = [j[0] for j in top[:ntop]]
        # Print out the topn songs for each topic and return their indices as a dictionary for further analysis:
        print("Topic " + str(i))
        print(music_df[['title','year','artist']].loc[topn_songs_by_topic[i]])
        print("*******************************")
    return topn_songs_by_topic

In [23]:
def json_to_tweet_text(json_lst):
    df = pd.DataFrame(json_lst)
    df_eng = df[df['lang']=='en']
    tweet_text = df_eng['full_text'].apply(get_tokens)
    return tweet_text


In [26]:
mar_text = json_to_tweet_text(mar_json)

In [27]:
relative_vocab = pd.read_csv('relatedQueries.csv')
relative_words = list(relative_vocab['word'])

In [28]:
print(relative_words)

['the prayer', 'prayer times', 'prayer time', 'catholic prayer', 'healing prayer', 'serenity prayer', 'serenity', 'what is prayer', 'lords prayer', 'morning prayer', 'prayer for healing', 'jesus prayer', 'daily prayer', 'our father prayer', 'our father', 'prayer quotes', 'muslim prayer', 'lord prayer', 'prayer plant', 'the lords prayer', 'islamic prayer', 'hail mary', 'hail mary prayer', 'house of prayer', 'night prayer']


In [ ]:
rising_vocab = pd.read_csv('risingQueries.csv')
rising_words = list(rising_vocab['word'])

In [29]:
#Measure religiosity
def relig_perc(text, vocab):
    relig_count = 0
    for word in vocab:
        relig_count += text.count(word)
    return relig_count/len(text)

In [31]:
mar_relig = mar_text.apply(lambda x: relig_perc(x, relative_words))

In [32]:
print(mar_relig.mean())

0.0


# Extracting the Geocodes

In [ ]:
from geopy.geocoders import OpenMapQuest
from geopy.extra.rate_limiter import RateLimiter
geolocator = OpenMapQuest(api_key='FGtjwBYCTEadGZpAIEkM2ySw5518bS7b') 
geocoder = RateLimiter(geolocator.geocode, min_delay_seconds=1) 
fwp_df['location'] = fwp_df['userlocation'].apply(geocoder)